# Reading Avro Blobs Into Parquet Data Sets

In [ ]:
import os
import string
import json
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when, date_sub, avg, window
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType, TimestampType, StructType, StringType, StructField
from pyspark.storagelevel import StorageLevel
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = os.getenv('TELEMETRY_STORAGE_ACCOUNT_NAME')
TELEMETRY_STORAGE_ACCOUNT_KEY = os.getenv('TELEMETRY_STORAGE_ACCOUNT_KEY')
TELEMETRY_CONTAINER_NAME = os.getenv('TELEMETRY_CONTAINER_NAME')
LOGS_ARCHIVE_CONTAINER_NAME = 'logs-archive' #TODO: introduce environment variables
STAGING_STORAGE_ACCOUNT_NAME = os.getenv('STAGING_STORAGE_ACCOUNT_NAME')
STAGING_STORAGE_ACCOUNT_KEY = os.getenv('STAGING_STORAGE_ACCOUNT_KEY')


In [ ]:
from pathlib import Path
data_dir = str(Path.home()) + '/data'

#TODO: Convert data_dir into env variable
% rm -rf $data_dir
% mkdir $data_dir $data_dir/logs

In [ ]:
wasbTelemetryUrl = "wasb://{0}@{1}.blob.{2}/*/*/*/*/*/*/*".format(TELEMETRY_CONTAINER_NAME, 
                                                                  TELEMETRY_STORAGE_ACCOUNT_NAME, 
                                                                  STORAGE_ACCOUNT_SUFFIX)
sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")
if TELEMETRY_STORAGE_ACCOUNT_KEY:
     hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(TELEMETRY_STORAGE_ACCOUNT_NAME), TELEMETRY_STORAGE_ACCOUNT_KEY)
hc.set("fs.azure.account.key.{}.blob.core.windows.net"
    .format(STAGING_STORAGE_ACCOUNT_NAME), STAGING_STORAGE_ACCOUNT_KEY)
sql = SQLContext.getOrCreate(sc)
avroblob = sql.read.format("com.databricks.spark.avro").load(wasbTelemetryUrl)
avroblob.show()

In [ ]:
#Convert byteformat to string format in pyspark dataframe
from json import loads as Loads
column = avroblob['Body']
string_udf = udf(lambda x: x.decode("utf-8"))
avroblob=avroblob.withColumn("BodyString", string_udf(column))
avroblob.printSchema()

In [ ]:
#Convert dataframe into 
new_df = sql.read.json(avroblob.select("BodyString").rdd.map(lambda r: r.BodyString))
new_df.show()

In [ ]:
new_df.write.parquet(data_dir+"/telemetry", mode="overwrite")